In [12]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [13]:
FLDR='drive/MyDrive/dogs-vs-cats/'

In [14]:
import os

In [15]:
files_zip_full=os.listdir(FLDR)

In [16]:
files_zip_full

['sampleSubmission.csv',
 'test1.zip',
 'train.zip',
 'weights.h5',
 'extracted',
 'extracted_2',
 'Model.h5']

In [17]:
training_data=files_zip_full[4]+'/train/'

In [18]:
training_data

'extracted/train/'

In [ ]:
Labels=[]
Names=[]

for f_n in os.listdir(FLDR+training_data):
  print(f_n)
  if 'cat' in f_n or 'Cat' in f_n:
    Names.append(f_n)
    Labels.append('cat')
  if 'dog' in f_n or 'Dog' in f_n:
    Names.append(f_n)
    Labels.append('dog')

In [20]:
import pandas as pd

In [33]:
from keras.applications.resnet_v2 import preprocess_input

In [21]:
df_train=pd.DataFrame(list(zip(Names,Labels)),columns=['Files','Labels'])

In [23]:
df_train.tail()

,Files,Labels
24995,cat.10893.jpg,cat
24996,cat.10894.jpg,cat
24997,cat.10895.jpg,cat
24998,cat.10896.jpg,cat
24999,cat.10897.jpg,cat


In [24]:
df_train['Labels'].value_counts()

dog    12500
cat    12500
Name: Labels, dtype: int64

In [48]:
from sklearn.model_selection import train_test_split
train,val= train_test_split(df_train, test_size=0.1)

In [49]:
train.to_csv(FLDR+'Train.csv',index=False)
val.to_csv(FLDR+'Validate.csv',index=False)

In [50]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [51]:
train_set=ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.1,
    shear_range=0.2,
    preprocessing_function=preprocess_input
)

In [52]:
val_set=ImageDataGenerator(preprocessing_function=preprocess_input)

In [53]:
train_gen=train_set.flow_from_dataframe(
    train,
    directory=FLDR+'extracted/train/',
    x_col='Files',
    y_col='Labels',
    batch_size=16,
    class_mode='binary',
    target_size=(224, 224),
)

Found 22500 validated image filenames belonging to 2 classes.


In [54]:
val_gen=val_set.flow_from_dataframe(
    val,
    directory=FLDR+'extracted/train/',
    x_col='Files',
    y_col='Labels',
    batch_size=16,
    class_mode='binary',
    target_size=(224, 224),
)

Found 2500 validated image filenames belonging to 2 classes.


In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, Input
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from keras import optimizers
from keras.callbacks import ModelCheckpoint

In [56]:
def model(shape=(224,224,3)):
  input_tensor=Input(shape=shape)             

  base_model = ResNet50V2(weights="imagenet", include_top=False, input_tensor=input_tensor) 
  model=Sequential()
  model.add(base_model)
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.3))
  model.add(Dense(1, activation='sigmoid'))

  for layer in model.layers:
    layer.trainable = True
  model.compile(optimizer=optimizers.Adam(lr=0.01), loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [57]:
Model=model()

In [58]:
Model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               51380736  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 513       
Total params: 74,946,049
Trainable params: 74,900,609
Non-trainable params: 45,440
_________________________________________________________________


In [ ]:
checkpoint = ModelCheckpoint(FLDR+"Model.h5", monitor='val_accuracy', save_best_only=True, mode='max') #creating checkpoint to save the best validation accuracy
callbacks_list = [checkpoint]
history = Model.fit_generator(train_gen, validation_data = val_gen, steps_per_epoch=50, callbacks=callbacks_list,verbose=1,epochs = 20)